# Step 2b: Data Cleaning, Pt. 3

At this point, I am working with the following datasets: 
1) For Grocery Market Data: The U.S. Department of Agriculture Agricultural Marketing Service's (AMS) [Market News Report](https://marketnews.usda.gov/mnp/dataDownload"), cleaned and filtered to just 2023 data. 
2) For Inflation Trend Data: The Bureau of Labor Statistics' Consumer Price Index (CPI) for [All Urban Consumers](https://www.bls.gov/cpi/tables/supplemental-files/), cleaned and filtered to just 2023 data.
3) For Verified Grocery Sales Data: The [Costco dataset](https://www.kaggle.com/datasets/bhavikjikadara/grocery-store-dataset) from Kaggle, which reported 2023 data, cleaned. 
4) For Regional & Demographic Data: I have switched from the[U.S. Census Dataset : Education, Finance, Industry](https://www.kaggle.com/datasets/mittvin/u-s-census-dataset-education-finance-industry) from Kaggle, which reported 2019-2020 data, to the U.S. Census dataset from the [Population Estimate Program (PEP)](https://www.census.gov/data/datasets/time-series/demo/popest/2020s-national-total.html), which reports 2020-2025 data and is therefore more useful to this analyis.  I may still try to incorporate education, finance, and industry data, though, if possible. 

I am going begin this portion of the cleaning and preparation process by looking at the dataset that handles product names the most usefully: The Bureau of Labor Statistics' Consumer Price Index (CPI) for [All Urban Consumers](https://www.bls.gov/cpi/tables/supplemental-files/).  I am going to see if I can get a reasonable list of proudcts to find comparables to in the other datasets.  The product column in each dataset, then, can be used as a foreign key from which to generate deeper insights by relating tables. 

In [33]:
# import libraries

import pandas as pd
pd.set_option('display.max_rows', None)

## A. Inflation Trend Data

In [34]:
# import cleaned BLS CPI dataset

bls_df = pd.read_csv('cleaned_bls_cpi_data.csv')

bls_df.head()

,product,unadjusted_percent_change_2023,seasonally_adjusted_effect_2023,unadjusted_effect_2023
0,All items,3.4,NaN,NaN
1,Food,2.7,0.028,0.366
2,Food at home,1.3,0.012,0.114
3,Cereals and bakery products,2.6,-0.003,0.030
4,Cereals and cereal products,0.4,-0.005,0.001


In [35]:
# How many unique products are listed?

bls_df['product'].nunique()

402

In [36]:
# Are there any duplicates? 

duplicates = bls_df[bls_df.duplicated(subset=['product'])]

print(duplicates)

# No duplicates

Empty DataFrame
Columns: [product, unadjusted_percent_change_2023, seasonally_adjusted_effect_2023, unadjusted_effect_2023]
Index: []


In [37]:
# I need to get my head around which products to keep and which to cut.
# What's the full list of products contained in this dataset? 

product_list = bls_df['product'].tolist()

print(product_list)

# It appears there's a lot of non-food items in this dataset.  Need to get those out. 

['All items', 'Food', 'Food at home', 'Cereals and bakery products', 'Cereals and cereal products', 'Flour and prepared flour mixes', 'Breakfast cereal(4)', 'Rice, pasta, cornmeal', 'Rice(4)(5)(6)', 'Bakery products(4)', 'Bread(4)(5)', 'White bread(4)(6)', 'Bread other than white(4)(6)', 'Fresh biscuits, rolls, muffins(5)', 'Cakes, cupcakes, and cookies(4)', 'Cookies(4)(6)', 'Fresh cakes and cupcakes(4)(6)', 'Other bakery products', 'Fresh sweetrolls, coffeecakes, doughnuts(4)(6)', 'Crackers, bread, and cracker products(6)', 'Frozen and refrigerated bakery products, pies, tarts, turnovers(6)', 'Meats, poultry, fish, and eggs', 'Meats, poultry, and fish', 'Meats', 'Beef and veal', 'Uncooked ground beef(4)', 'Uncooked beef roasts(5)', 'Uncooked beef steaks(5)', 'Uncooked other beef and veal(4)(5)', 'Pork', 'Bacon, breakfast sausage, and related products(5)', 'Bacon and related products(6)', 'Breakfast sausage and related products(5)(6)', 'Ham', 'Ham, excluding canned(6)', 'Pork chops(4)'

In [38]:
bls_df

# Row 119 is the last one with food-related data.  Slice everything else out.

bls_df = bls_df[:119]

bls_df

,product,unadjusted_percent_change_2023,seasonally_adjusted_effect_2023,unadjusted_effect_2023
0,All items,3.4,NaN,NaN
1,Food,2.7,0.028,0.366
2,Food at home,1.3,0.012,0.114
3,Cereals and bakery products,2.6,-0.003,0.030
4,Cereals and cereal products,0.4,-0.005,0.001
5,Flour and prepared flour mixes,2.2,0.000,0.001
6,Breakfast cereal(4),0.3,-0.004,0.000
7,"Rice, pasta, cornmeal",-0.3,0.000,0.000
8,Rice(4)(5)(6),0.1,NaN,NaN
9,Bakery products(4),3.6,-0.003,0.028


In [39]:
# Prune the proudcts column down to only wanted items. 

indexes_to_keep = [1, 2, 5, 6, 8, 9, 10, 15, 24, 25, 29, 31, 32, 33, 35, 40, 41, 42, 45, 46, 49, 50, 52, 55, 56, 60, 61, 62, 63, 66, 67, 68, 69, 74, 76, 78, 85, 96, 100, 113, 118]

bls_df = bls_df.loc[indexes_to_keep].reset_index(drop=True)

bls_df

,product,unadjusted_percent_change_2023,seasonally_adjusted_effect_2023,unadjusted_effect_2023
0,Food,2.7,0.028,0.366
1,Food at home,1.3,0.012,0.114
2,Flour and prepared flour mixes,2.2,0.000,0.001
3,Breakfast cereal(4),0.3,-0.004,0.000
4,Rice(4)(5)(6),0.1,NaN,NaN
5,Bakery products(4),3.6,-0.003,0.028
6,Bread(4)(5),3.1,-0.001,0.007
7,Cookies(4)(6),2.7,NaN,NaN
8,Beef and veal,8.7,0.001,0.038
9,Uncooked ground beef(4),6.7,-0.001,0.011


In [40]:
# Looking further at it, I think unadjusted_percent_change is the numeric data I'm going to want to use from this table. 
# Drop the others. 

bls_df.drop(columns=['seasonally_adjusted_effect_2023', 'unadjusted_effect_2023'], axis=1, inplace=True)

bls_df

,product,unadjusted_percent_change_2023
0,Food,2.7
1,Food at home,1.3
2,Flour and prepared flour mixes,2.2
3,Breakfast cereal(4),0.3
4,Rice(4)(5)(6),0.1
5,Bakery products(4),3.6
6,Bread(4)(5),3.1
7,Cookies(4)(6),2.7
8,Beef and veal,8.7
9,Uncooked ground beef(4),6.7


In [41]:
# Strip parenthetical information from product names

bls_df['product'] = bls_df['product'].str.replace(r'\(.*\)', '', regex=True)

bls_df

,product,unadjusted_percent_change_2023
0,Food,2.7
1,Food at home,1.3
2,Flour and prepared flour mixes,2.2
3,Breakfast cereal,0.3
4,Rice,0.1
5,Bakery products,3.6
6,Bread,3.1
7,Cookies,2.7
8,Beef and veal,8.7
9,Uncooked ground beef,6.7


In [115]:
# Normalize product names

rename_dict = {
    'Flour and prepared flour mixes' : 'Flour',
    'Breakfast cereal' : 'Cereal', 
    'Bakery products' : 'Bakery', 
    'Uncooked ground beef' : 'Ground beef', 
    'Bacon and related products' : 'Bacon', 
    'Breakfast sausage and related products' : 'Bkfst sausage', 
    'Fresh whole chicken' : 'Whole chicken', 
    'Fish and seafood' : 'Fish', 
    'Fresh fish and seafood' : 'Fresh fish', 
    'Frozen fish and seafood' : 'Frozen fish', 
    'Cheese and related products' : 'Cheese', 
    'Ice cream and related products' : 'Ice cream', 
    'Dried beans, peas, and lentils' : 'Beans'
}

bls_df['product'] = bls_df['product'].replace(rename_dict)

bls_df

,product,unadjusted_percent_change_2023
0,food,2.7
1,food at home,1.3
2,flour,2.2
3,cereal,0.3
4,rice,0.1
5,bakery,3.6
6,bread,3.1
7,cookies,2.7
8,beef and veal,8.7
9,ground beef,6.7


### Takeaways:
* That's starting to look like a reasonable dataset to use to find matching products in the other datasets. 
* Now... how to do that...

## B. Grocery Market Data

In [45]:
usda_df = pd.read_csv('cleaned_usda_data_grouped_and_filtered.csv')

usda_df.head()

,product_name,region,report_date,mean_weighted_price,year
0,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-02-03,3.99,2023
1,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-02-17,5.99,2023
2,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-03-31,4.98,2023
3,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-05-26,4.98,2023
4,APPLES BRAEBURN 3 lb bag,NATIONAL,2023-02-03,3.99,2023


In [ ]:
# Let's take 'apples' as the first test case, here. 
# What results can I find for 'apples' in this dataset?

apples_matches = usda_df[usda_df['product_name'].str.contains('apples', case=False, na=False)]

apples_matches

# So... a product like 'apples' returned over 3000 rows of data.  This isn't going to be useful,
# particularly given how many *different* ways apples are listed (variety, packaging, etc.).  I
# need to get this down to ONE row of data per product (i.e. 'apple'), per region, per report date.  

,product,region,report_date,mean_weighted_price
0,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-02-03,3.99
1,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-02-17,5.99
2,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-03-31,4.98
3,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-05-26,4.98
4,APPLES BRAEBURN 3 lb bag,NATIONAL,2023-02-03,3.99


In [ ]:
# First, let's get the year column out of there (they're all 2023) and rename product_name
# to match the BLS CPI dataset. 

usda_df.drop('year', axis=1, inplace=True)

usda_df.rename(columns={'product_name': 'product'}, inplace=True)

usda_df.head()

,product,region,report_date,mean_weighted_price
0,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-02-03,3.99
1,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-02-17,5.99
2,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-03-31,4.98
3,APPLES BRAEBURN 3 lb bag,MIDWEST U.S.,2023-05-26,4.98
4,APPLES BRAEBURN 3 lb bag,NATIONAL,2023-02-03,3.99


In [ ]:
# Okay... now... from the top. First up: flour. 

matches = usda_df[usda_df['product'].str.contains('peanut', case=False, na=False)]

matches

# Apparently there are none?  Flour = out. What else ISN'T in the USDA dataset? 

# Flour, cereal, rice, bakery, bread, cookies, poultry, whole chicken, fish, fresh fish, frozen fish,
# eggs, milk, cheese, ice cream, fresh fruits, bananas, citrus fruits (orange is present), fresh
# vegetables, canned vegetables, frozen vegetables, 'beans, peas, lentils' only as 'beans', coffee, 
# butter, peanut butter' 

,product,region,report_date,mean_weighted_price


### QUESTION: 
* Am I only going to keep products that are found in all three of the grocery datasets (USDA, BLS, and Costco)?  Or, I could keep matches in USDA and BLS, and again BLS and Costco.  What about matches between USDA and Costco datasets, too?
* I think I need to stick to using the BLS dataset as a starting point.  

In [ ]:
# Filter the usda_df to only include products also found in the bls_df. 

# Ensure lowercase and strip spaces
bls_df['product'] = bls_df['product'].str.lower().str.strip()
usda_df['product'] = usda_df['product'].str.lower().str.strip()

# Create regex pattern to match any BLS product name within USDA product names
pattern = '|'.join(bls_df['product'].tolist())  # Join all words with '|' for OR matching

# Filter USDA dataset where the product column contains any BLS product name
filtered_usda_df = usda_df[usda_df['product'].str.contains(pattern, case=False, na=False)]

filtered_usda_df.shape

# I think the usda_df was at least 10x that size before.  We're getting somehwere here! 


(26013, 4)

In [113]:
# Now, I need to clean up each product still remaining in the usda_df. 
# I need a list of products that matched in the usda_df from the bls_df. 

# Initialize an empty list to store matched product names
matched_products = []

# Loop over each product in bls_df
for product in bls_df['product']:
    # Check if the product from bls_df is found anywhere in usda_df
    if usda_df['product'].str.contains(product, case=False, na=False).any():
        matched_products.append(product)

# Display the list of matched products
matched_products

['ground beef',
 'pork',
 'bacon',
 'ham',
 'chicken',
 'apples',
 'potatoes',
 'lettuce',
 'tomatoes',
 'butter']

In [125]:
# I think there's a few more that should've matched, but didn't due to formatting errors.

rename_dict = {
    'breakfast sausage' : 'sausage bkfst', 
    'dried beans, peas, and lentils' : 'beans'
}

bls_df['product'] = bls_df['product'].replace(rename_dict)

bls_df


,product,unadjusted_percent_change_2023
0,food,2.7
1,food at home,1.3
2,flour,2.2
3,cereal,0.3
4,rice,0.1
5,bakery,3.6
6,bread,3.1
7,cookies,2.7
8,beef and veal,8.7
9,ground beef,6.7


In [135]:
# Re-run matching code

# Create regex pattern to match any BLS product name within USDA product names
pattern = '|'.join(bls_df['product'].tolist())  # Join all words with '|' for OR matching

# Filter USDA dataset where the product column contains any BLS product name
usda_df = usda_df[usda_df['product'].str.contains(pattern, case=False, na=False)]

usda_df.shape

(26246, 4)

In [136]:
# List matches again

# Initialize an empty list to store matched product names
matched_products = []

# Loop over each product in bls_df
for product in bls_df['product']:
    # Check if the product from bls_df is found anywhere in usda_df
    if usda_df['product'].str.contains(product, case=False, na=False).any():
        matched_products.append(product)

# Display the list of matched products
matched_products

['ground beef',
 'pork',
 'bacon',
 'ham',
 'chicken',
 'apples',
 'potatoes',
 'lettuce',
 'tomatoes',
 'beans',
 'butter']

In [137]:
# Okay... let's get to work, starting with 'ground beef' 

ground_beef = usda_df[usda_df['product'].str.contains(r'ground beef', case=False, na=False)]
                      
ground_beef.head()

,product,region,report_date,mean_weighted_price
11542,beef ground beef 70-79% per pound,ALASKA,2023-02-03,2.49
11543,beef ground beef 70-79% per pound,ALASKA,2023-03-03,2.49
11544,beef ground beef 70-79% per pound,HAWAII,2023-01-06,4.69
11545,beef ground beef 70-79% per pound,HAWAII,2023-01-20,4.59
11546,beef ground beef 70-79% per pound,HAWAII,2023-01-27,4.59


In [ ]:
# Looking at this and thinking about how I'll want to analyze / visualize, it seems like having
# one data point per MONTH rather than per WEEK per region is what I want. So... let's make that.

# Ensure the report_date is in datetime format if not already
usda_df['report_date'] = pd.to_datetime(usda_df['report_date'])

# Extract year and month from report_date
usda_df['year_month'] = usda_df['report_date'].dt.to_period('M')

# Group by product, region, and the new year_month, and aggregate the price (mean in this case)
usda_df = usda_df.groupby(['product', 'region', 'year_month'], as_index=False)['mean_weighted_price'].mean()

usda_df = usda_df.drop_duplicates(subset=['product', 'region', 'year_month'])

usda_df.head()

usda_df.shape

(8177, 4)

In [ ]:
# Okay... back to ground beef.  I don't need things like 70-79%, 80-89%, etc. because
# the bls_df just lists 'ground beef' as a product. I want to aggregate a mean price 
# for all varieties of ground beef.

# Clean the 'product' column to only contain the base product name 'ground beef'

usda_df['product_name'] = usda_df['product'].str.extract(r'(ground beef)')

# # Group by 'product_name', 'region', and 'report_date', then calculate the mean price
usda_df = usda_df.groupby(['product_name', 'region', 'year_month'])['mean_weighted_price'].mean().reset_index()

usda_df.head(25)

# Oh my gosh, I think I did it... 


,product_name,region,year_month,mean_weighted_price
0,ground beef,ALASKA,2023-01,4.751250
1,ground beef,ALASKA,2023-02,3.769583
2,ground beef,ALASKA,2023-03,4.455833
3,ground beef,ALASKA,2023-04,4.621250
4,ground beef,ALASKA,2023-05,4.839375
5,ground beef,ALASKA,2023-06,5.943500
6,ground beef,ALASKA,2023-07,4.336250
7,ground beef,ALASKA,2023-08,5.328750
8,ground beef,ALASKA,2023-09,5.017000
9,ground beef,ALASKA,2023-10,5.191875
